In [1]:
import numpy as np
#import xml.etree.ElementTree as ET
from lxml import objectify

In [8]:
# I don't want to have to redo all the smarts definitions
with open("uff-desc.xml", "r") as f:
    data=f.readlines()
xml_str = "".join(data)
root = objectify.fromstring(xml_str)

uff_dict = {}
for a in root.AtomTypes[0].Type:
    #print(a.attrib)
    try:
        defn = a.attrib['def']
    except KeyError:
        defn = None
    try:
        desc = a.attrib['desc']
    except KeyError:
        desc = None
    try:
        overrides = a.attrib['overrides']
    except KeyError:
        overrides = None
    a_dict = {
        "definition" : defn,
        "description" : desc,
        "overrides" : overrides
    }
    uff_dict[a.attrib["name"]] = a_dict

In [4]:
# This txt file is Table 1 from Rappe 1992. I could not find a plain text version of it,
# so it is copied from PDF which is a character recognition nightmare. 
# I have checked it with most focus going towards atom types I want to use,
# but errors probably exist! D: 

# angstroem, deg, kcal/mol, charge
# type, element, bond, angle, distance, energy, scale, charge
d_uff = np.dtype([
    ("type", np.unicode_, 8), 
    ("element", np.unicode_, 2), 
    ("bond", "d"),
    ("angle", "d"),
    ("distance", "d"),
    ("energy", "d"),
    ("scale", "d"),
    ("charge", "d"),
])
uff_data = np.loadtxt("uff.txt", dtype=d_uff)
#print(uff_data[0]["type"])

In [5]:
mass_dict = {}
with open("masses.txt") as f:
    for line in f.readlines():
        elem, mass = line.split()
        mass_dict[elem] = mass
#print(mass_dict)

In [19]:
with open("uff-gmso.xml", "w") as f:
    f.write('<ForceField version="0.0.1" name="UFF">\n\t<FFMetaData>\n')
    f.write('\t\t<Units energy="kcal/mol" distance="angstrom" mass="amu" charge="qe"/>\n')
    f.write('\t</FFMetaData>\n\t<AtomTypes expression="4*epsilon*((sigma/r)**12 - (sigma/r)**6)">\n')
    f.write('\t\t<ParametersUnitDef parameter="sigma" unit="angstrom"/>\n')
    f.write('\t\t<ParametersUnitDef parameter="epsilon" unit="kcal/mol"/>\n')
    for atom in uff_data:
        a_str = ""
        for param in ["definition", "overrides", "description"]:
            try:
                val = uff_dict[atom["type"]][param]
            except KeyError:
                val = None
            if val != None:
                a_str += f' {param} = "{val}"'
        try:
            mass = mass_dict[atom["element"]]
        except KeyError:
            continue # if we don't have the mass-- don't write this atom
        f.write(f'\t\t<AtomType name="{atom["type"]}" element="{atom["element"]}" mass="{mass}"{a_str}>\n')
        f.write('\t\t\t<Parameters>\n')
        f.write(f'\t\t\t\t<Parameter name="sigma" value="{atom["distance"]}"/>\n')
        f.write(f'\t\t\t\t<Parameter name="epsilon" value="{atom["energy"]}"/>\n')
        f.write('\t\t\t</Parameters>\n')
        f.write('\t\t</AtomType>\n')

    f.write('\t</AtomTypes>\n')
    f.write('\t<BondTypes expression="1/2*k*(r-rij)**2">\n')
    f.write('\t\t<ParametersUnitDef parameter="rij" unit="angstrom"/>\n')
    f.write('\t\t<ParametersUnitDef parameter="k" unit="(kcal/mol)/angstrom**2"/>\n')
    
    for i,atom1 in enumerate(uff_data):
        for atom2 in uff_data[i:]:
            try:
                mass_dict[atom1["element"]]
                mass_dict[atom2["element"]]
            except KeyError:
                continue
            # so in the rappe paper r_ij is supposed to be 
            # r_ij = r_i + r_j +r_BO + r_EN 
            # but to simplify things, I'm going to try just 
            # r_ij = r_i + r_j + 0.026
            # 0.026 is a correction so the c-c bond comes out right.
            length = atom1["bond"] + atom2["bond"] + 0.026 #angstroem
            
            # k_ij = 664.12 (Z*_i x Z*_j)/(r_ij^3)
            energy = 664.12 * atom1["charge"] * atom2["charge"]/ length**3
            name = atom1["type"] + atom2["type"] # this was necessary to stop error
            f.write(f'\t\t<BondType name="{name}" type1="{atom1["type"]}" type2="{atom2["type"]}">\n')
            f.write('\t\t\t<Parameters>\n')
            f.write(f'\t\t\t\t<Parameter name="rij" value="{length}"/>\n')
            f.write(f'\t\t\t\t<Parameter name="k" value="{energy}"/>\n')
            f.write('\t\t\t</Parameters>\n')
            f.write('\t\t</BondType>\n')

    f.write('\t</BondTypes>\n')
    f.write('</ForceField>')

if you don't give BondType a name, you get this error--even though name should be optional according to the [schema](https://github.com/mosdef-hub/gmso/blob/af40af05f22a36b6d7fcc076a8306d4cc2179a46/gmso/utils/schema/ff-gmso.xsd)

```DocumentInvalid: Element 'BondType': Not all fields of key identity-constraint 'bondtype_name_key' evaluate to a node., line 708 xml```